<a href="https://colab.research.google.com/github/u-ryu-00/machine-learning/blob/main/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# 코드 출처 - https://github.com/rickiepark/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/09_transformer/gpt/gpt.ipynb

import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

In [2]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

In [3]:
import sys

# 코랩일 경우 노트북에서 celeba 데이터셋을 받습니다.
if 'google.colab' in sys.modules:
    # 캐글-->Setttings-->API-->Create New Token에서
    # kaggle.json 파일을 만들어 코랩에 업로드하세요.
    from google.colab import files
    files.upload()
    !mkdir ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    # celeba 데이터셋을 다운로드하고 압축을 해제합니다.
    !kaggle datasets download -d zynicide/wine-reviews
    !unzip -q wine-reviews.zip
    # model 디렉토리를 만듭니다.
    !mkdir models

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
License(s): CC-BY-NC-SA-4.0
100% 50.9M/50.9M [00:03<00:00, 17.1MB/s]
100% 50.9M/50.9M [00:03<00:00, 14.8MB/s]


In [4]:
# 전체 데이터셋 로드
with open("./winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

In [ ]:
wine_data[10]

{'points': '87',
 'title': 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)',
 'description': 'Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.',
 'taster_name': 'Virginie Boone',
 'taster_twitter_handle': '@vboone',
 'price': 19,
 'designation': 'Mountain Cuvée',
 'variety': 'Cabernet Sauvignon',
 'region_1': 'Napa Valley',
 'region_2': 'Napa',
 'province': 'California',
 'country': 'US',
 'winery': 'Kirkland Signature'}

In [5]:
# 데이터셋 필터링
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]

In [6]:
# 레시피 개수 카운트
n_wines = len(filtered_data)
print(f"{n_wines}개 레시피 로드")

129907개 레시피 로드


In [7]:
example = filtered_data[25]
print(example)

wine review : US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site. Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture.


In [8]:
# 구두점 앞에 공백을 채워서 별도의 '단어'로 취급합니다.
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [9]:
# 레시피 예시 표시
example_data = text_data[25]
example_data

'wine review : US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard - designated Pinot that hails from a high - elevation site . Small in production , it offers intense , full - bodied raspberry and blackberry steeped in smoky spice and smooth texture . '

In [10]:
# 텐서플로 데이터셋으로 변환하기
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# TextVectorization 층 만들기
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# 훈련 세트에 이 층을 적용합니다.
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [13]:
# 토큰:단어 매핑 표시
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


In [14]:
# 동일한 샘플을 정수로 변환하여 표시
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[   7   10    2   20    2   29    2   43   62    2   55    5  243 4145
  453  634   26    9  497  499  667   17   12  142   14 2214   43   25
 2484   32    8  223   14 2213  948    4  594   17  987    3   15   75
  237    3   64   14   82   97    5   74 2633   17  198   49    5  125
   77    4    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


In [15]:
# 레시피 텍스트와 한 단어만큼 이동된 동일한 텍스트로 훈련 세트를 만듭니다.
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [16]:
example_input_output = train_ds.take(1).get_single_element()

In [17]:
# 입력 예시
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   7,   10,    2,   42,    2,  513,   42,    2, 1089,   14,   92,
          2,    1,   18,   21,  186,  292,    7,    3,  256,  227,    6,
         96,  398,    9, 1089,    4,   15,   41,   38,   70,    3,   76,
         30,    5,   34,    5,    8,  147,    9,   49,    4,   12,   13,
          8,  183,    7,   46, 4940,   11,   54,  186, 3393,  110,  366,
         88,    6, 1718,    4,   35,   66,    4,  802,    4,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [18]:
# 출력 예시(토큰 하나씩 이동)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,    2,   42,    2,  513,   42,    2, 1089,   14,   92,    2,
          1,   18,   21,  186,  292,    7,    3,  256,  227,    6,   96,
        398,    9, 1089,    4,   15,   41,   38,   70,    3,   76,   30,
          5,   34,    5,    8,  147,    9,   49,    4,   12,   13,    8,
        183,    7,   46, 4940,   11,   54,  186, 3393,  110,  366,   88,
          6, 1718,    4,   35,   66,    4,  802,    4,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [19]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

In [20]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

In [21]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [22]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [23]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 256)           │       2,580,480 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ [(None, None, 256), (None,  │         658,688 │
│                                      │ 2, None, None)]             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, None, 10000)         │       2,570,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

In [25]:
# TextGenerator 체크포인트 만들기
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\n생성된 텍스트:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [26]:
# 모델 저장 체크포인트 만들기
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# 시작 프롬프트 토큰화
text_generator = TextGenerator(vocab)

In [27]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
4055/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.5932
생성된 텍스트:
wine review : italy : veneto : glera : pleasantly [UNK] ' s crown translates their smile to the son , but has is more touches of banana and kiwi fruit on the nose and the wine . below the palate is fresh , with ripe notes of mature apple marmalade , candied hazelnut and lemon drop . 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 350s 78ms/step - loss: 2.5927
Epoch 2/5
4057/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.9841
생성된 텍스트:
wine review : us : california : sauvignon blanc : this is a cool vintage brand , making that was fermented heavily on for up in french oak . it has some browning in color and bristling acidity , despite its high - toned , layering of lemony , flirty flavors . 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 1.9841
Epoch 3/5
4056/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.8943
생성된 텍스트:
wine review : italy : sicily & sardinia : grillo : inzolia - based aromas of crushed stone , passion

In [28]:
# 최종 모델 저장
gpt.export("./models/gpt_saved_model")

Saved artifact at './models/gpt_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None), dtype=tf.int32, name='keras_tensor')
Output Type:
  List[TensorSpec(shape=(None, None, 10000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2, None, None), dtype=tf.float32, name=None)]
Captures:
  131953519790912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953525728080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953525732480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953519946992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953519940304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953519938896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953519951216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953519945760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953519937136: TensorSpec(shape=(), dtype=tf.resource, name=None)
 

In [29]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [30]:
info = text_generator.generate(
    "wine review : us", max_tokens=80, temperature=1.0
)


생성된 텍스트:
wine review : us : washington : cabernet sauvignon : this wine is well rendered woodspice , with lighter - styled aromas of cherry fruit . it ' s full bodied and chalky in feel with bright fruit flavors , trailing toward the finish . 



In [31]:
info = text_generator.generate(
    "wine review : italy", max_tokens=80, temperature=0.5
)


생성된 텍스트:
wine review : italy : tuscany : sangiovese grosso : this is a big , masculine expression of sangiovese ( with loads of spice and spice . the bouquet is followed by flavors of blackberry , leather , tobacco , vanilla and tobacco . 



In [32]:
info = text_generator.generate(
    "wine review : germany", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


생성된 텍스트:
wine review : germany : mosel : riesling : hints of smoke and earth add complexity to this dry , full - bodied riesling . it ' s brisk and slightly bitter , with a juicy , refreshing finish . 



::   	100.0%
grosso:   	0.0%
-:   	0.0%
zealand:   	0.0%
[UNK]:   	0.0%
--------



mosel:   	71.14%
pfalz:   	14.03%
rheingau:   	11.46%
rheinhessen:   	1.45%
baden:   	1.07%
--------



::   	61.47%
-:   	38.52%
grosso:   	0.0%
[UNK]:   	0.0%
other:   	0.0%
--------



riesling:   	99.99%
pinot:   	0.0%
gewürztraminer:   	0.0%
spätburgunder:   	0.0%
grüner:   	0.0%
--------



...

,:   	95.35%
midpalate:   	0.81%
core:   	0.81%
vein:   	0.51%
finish:   	0.49%
--------



mineral:   	18.28%
fruity:   	9.55%
juicy:   	8.38%
lemony:   	7.16%
slightly:   	6.37%
--------



finish:   	74.95%
tone:   	4.22%
feel:   	2.29%
mineral:   	2.26%
character:   	2.15%
--------



.:   	99.97%
that:   	0.02%
,:   	0.01%
marked:   	0.0%
with:   	0.0%
--------



:   	96.86%
drink:   	2.65%
it:   	0.32%
enjoy:   	0.14%
the:   	0.02%
--------

